In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))


def train_classifier(features_train, features_test, label_train, label_test, classifier):
    if classifier == "Logistic_Regression":
        model = LogisticRegression(C=1.)
    elif classifier == "Naive_Bayes":
        model = MultinomialNB()
    elif classifier == "SVM":
        model = SVC()
    elif classifier == "Linear":
        model = LinearRegression()    
    elif classifier == "Random_Forest":
        model = RandomForestClassifier(n_estimators=400, random_state=11)
    elif classifier == "Kmeans":
        knn = neighbors.KNeighborsRegressor()
        params = {'n_neighbors':[2,3,4,5,6,7,8,9]}
        model = GridSearchCV(knn, params, cv=5)
    else:
        print("Incorrect Selection Of Classifier")

    model.fit(features_train, label_train)
    print("Model Fitting Done")

    fileName = './Sentiment_models/' + classifier + '.pickle'
    with open(fileName, 'wb') as file:
        pickle.dump(model, file)
    print("Pickle File Created %s" % fileName)

    accuracy = model.score(features_test, label_test)
    print("Accuracy Is:", accuracy)

    return model

In [34]:
#ID,Title,Air_Date,Production_Code,Season,Episode_No.,Total_Episodes_Till_Now,US_Viewers_In_Millions,Views,IMDB_Rating,IMDB_Votes,Image_URL,Video_URL,Retweets,Favorites,Vader_Score,Sentiment_Score,Tweets_Per_Day,Unique_Users
 
# fileName="simpsons_episodes.csv"
# my_df = pd.read_csv(fileName)

train = pd.read_csv("simpsons_episodes_train.csv")
test = pd.read_csv("simpsons_episodes_test.csv")


print("File read")
for i in ['ID','Title','Air_Date','Production_Code','Season','Episode_No.',
          'Total_Episodes_Till_Now','Image_URL','Video_URL']:
    # del my_df[i]
    del train[i]
    del test[i]
# print(my_df.head())
# my_df_temp =my_df   
     

File read


In [35]:
# my_df.dropna(inplace=True)
# my_df.reset_index(drop=True,inplace=True)
# my_df.info()
train.dropna(inplace=True)
train.reset_index(drop=True,inplace=True)
train.info()
test.dropna(inplace=True)
test.reset_index(drop=True,inplace=True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110 entries, 0 to 109
Data columns (total 10 columns):
US_Viewers_In_Millions    110 non-null float64
Views                     110 non-null float64
IMDB_Rating               110 non-null float64
IMDB_Votes                110 non-null float64
Retweets                  110 non-null float64
Favorites                 110 non-null float64
Vader_Score               110 non-null float64
Sentiment_Score           110 non-null float64
Tweets_Per_Day            110 non-null float64
Unique_Users              110 non-null float64
dtypes: float64(10)
memory usage: 8.7 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 10 columns):
US_Viewers_In_Millions    22 non-null float64
Views                     22 non-null float64
IMDB_Rating               22 non-null float64
IMDB_Votes                22 non-null float64
Retweets                  22 non-null float64
Favorites                 22 non-null float64
Vader

In [36]:
X = my_df.loc[:, ['Views','IMDB_Votes','IMDB_Rating','Vader_Score','Sentiment_Score'
                  ,'Retweets','Favorites','Tweets_Per_Day','Unique_Users']]

y = my_df.loc[:, ['US_Viewers_In_Millions']]

print(X.head())
print(y.head())


NameError: name 'my_df' is not defined

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=0)
# let's print shape of each train and testing
print("Shape of X_train: ", X_train.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_test", y_test.shape)

NameError: name 'X' is not defined

In [12]:
print("Model Training Started")
algorithm = "Linear"
model = train_classifier(X_train,X_test, y_train, y_test, algorithm)
print("Model Training Complete")


Model Training Started


NameError: name 'X_train' is not defined

In [37]:
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "99"

In [38]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6673168089471086769]

In [39]:
# Test with a simple computation
import tensorflow as tf

tf.Session()

with tf.device('/cpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3])
# If you have gpu you can try this line to compute b with your GPU
#with tf.device('/gpu:0'):    
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2])
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

print(sess.run(c))

# Runs the op.
# Log information
options = tf.RunOptions(output_partition_graphs=True)
metadata = tf.RunMetadata()
c_val = sess.run(c, options=options, run_metadata=metadata)

print(metadata.partition_graphs)

sess.close()

[[22. 28.]
 [49. 64.]]
[node {
  name: "MatMul_2"
  op: "Const"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
          dim {
            size: 2
          }
          dim {
            size: 2
          }
        }
        tensor_content: "\000\000\260A\000\000\340A\000\000DB\000\000\200B"
      }
    }
  }
  experimental_debug_info {
    original_node_names: "MatMul_2"
  }
}
node {
  name: "_retval_MatMul_2_0_0"
  op: "_Retval"
  input: "MatMul_2"
  device: "/job:localhost/replica:0/task:0/device:CPU:0"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "index"
    value {
      i: 0
    }
  }
  experimental_debug_info {
    original_node_names: "_retval_MatMul_2_0_0"
  }
}
library {
}
versions {
  producer: 27
}
]


In [40]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams
import matplotlib


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

tf.logging.set_verbosity(tf.logging.INFO)
sess = tf.InteractiveSession()

# train = pd.read_csv('../input/train.csv')
# train, test= train_test_split(my_df_temp, test_size = 0.1, random_state=0)

train.fillna(0,inplace=True)

test.fillna(0,inplace=True)

print('Shape of the train data with numerical features:', train.shape)
print('Shape of the test data with numerical features:', test.shape)


print("")
print("List of features contained our dataset:",list(train.columns))
print("List of features contained our dataset:",list(test.columns))

print("Number of rows: ", train.shape[0])
rows =train.shape[0]

Shape of the train data with numerical features: (110, 10)
Shape of the test data with numerical features: (22, 10)

List of features contained our dataset: ['US_Viewers_In_Millions', 'Views', 'IMDB_Rating', 'IMDB_Votes', 'Retweets', 'Favorites', 'Vader_Score', 'Sentiment_Score', 'Tweets_Per_Day', 'Unique_Users']
List of features contained our dataset: ['US_Viewers_In_Millions', 'Views', 'IMDB_Rating', 'IMDB_Votes', 'Retweets', 'Favorites', 'Vader_Score', 'Sentiment_Score', 'Tweets_Per_Day', 'Unique_Users']
Number of rows:  110


In [41]:
import warnings
warnings.filterwarnings('ignore')

col_train = list(train.columns)
print(col_train)
col_train_bis = list(train.columns)
print("nvjhv")
col_train_bis.remove('US_Viewers_In_Millions')
print(col_train_bis)
viewership_temp = train['US_Viewers_In_Millions']
mat_train = np.matrix(train)
mat_test  = np.matrix(test)
mat_new = np.matrix(test.drop('US_Viewers_In_Millions',axis = 1))
mat_y = np.array(train.US_Viewers_In_Millions).reshape((rows,1))
 
prepro_y = MinMaxScaler()
prepro_y.fit(mat_y)
print("Hello1")
prepro = MinMaxScaler()
prepro.fit(mat_train)
print("Hello1")
prepro_test = MinMaxScaler()
prepro_test.fit(mat_new)
print("Hello1")
train = pd.DataFrame(prepro.transform(mat_train),columns = col_train)
train['US_Viewers_In_Millions']= viewership_temp 

test  = pd.DataFrame(prepro_test.transform(mat_new),columns = col_train_bis)
print(train.head())
print(test.head())

['US_Viewers_In_Millions', 'Views', 'IMDB_Rating', 'IMDB_Votes', 'Retweets', 'Favorites', 'Vader_Score', 'Sentiment_Score', 'Tweets_Per_Day', 'Unique_Users']
nvjhv
['Views', 'IMDB_Rating', 'IMDB_Votes', 'Retweets', 'Favorites', 'Vader_Score', 'Sentiment_Score', 'Tweets_Per_Day', 'Unique_Users']
Hello1
Hello1
Hello1
   US_Viewers_In_Millions     Views  IMDB_Rating  IMDB_Votes  Retweets  \
0               8650000.0  0.167802     0.621622    0.130332       0.0   
1              14620000.0  0.267141     0.594595    0.214455       0.0   
2               5110000.0  0.283668     0.702703    0.190758       0.0   
3               5870000.0  0.386112     0.621622    0.182464       0.0   
4               6080000.0  0.288802     0.567568    0.148104       0.0   

   Favorites  Vader_Score  Sentiment_Score  Tweets_Per_Day  Unique_Users  
0        0.0     0.395580         0.000000        0.000000      0.000000  
1        0.0     0.396548         0.052632        0.007458      0.003247  
2        0.0 

In [43]:
# List of features
COLUMNS = col_train
FEATURES = col_train_bis
LABEL = "US_Viewers_In_Millions"

# Columns for tensorflow
feature_cols = [tf.contrib.layers.real_valued_column(k) for k in FEATURES]

# Training set and Prediction set with the features to predict
training_set = train[COLUMNS]
prediction_set = train.US_Viewers_In_Millions

# Train and Test 
x_train, x_test, y_train, y_test = train_test_split(training_set[FEATURES] , prediction_set, test_size=0.2, random_state=42)
y_train = pd.DataFrame(y_train, columns = [LABEL])
training_set = pd.DataFrame(x_train, columns = FEATURES).merge(y_train, left_index = True, right_index = True)
print(training_set.head())

# Training for submission
training_sub = training_set[col_train]
print("Done")

       Views  IMDB_Rating  IMDB_Votes  Retweets  Favorites  Vader_Score  \
65  0.278535     0.729730    0.144550       0.0        0.0     0.395580   
26  0.329669     0.675676    0.136256       0.0        0.0     0.395580   
22  0.356964     0.648649    0.228673       0.0        0.0     0.395580   
31  0.292382     0.540541    0.117299       0.0        0.0     0.395580   
47  0.344293     0.675676    0.215640       0.0        0.0     0.374128   

    Sentiment_Score  Tweets_Per_Day  Unique_Users  US_Viewers_In_Millions  
65         0.105263        0.115439      0.092175               7440000.0  
26         0.052632        0.053585      0.024571               6500000.0  
22         0.000000        0.039074      0.017236               9540000.0  
31         0.000000        0.067752      0.048597               6140000.0  
47         0.000000        0.164039      0.055750              11480000.0  
Done


In [44]:
# Same thing but for the test set
y_test = pd.DataFrame(y_test, columns = [LABEL])
testing_set = pd.DataFrame(x_test, columns = FEATURES).merge(y_test, left_index = True, right_index = True)
print(testing_set.head())

       Views  IMDB_Rating  IMDB_Votes  Retweets  Favorites  Vader_Score  \
78  0.323033     0.486486    0.046209       0.5        0.5      0.39558   
10  0.278109     0.648649    0.165877       0.0        0.0      0.39558   
4   0.288802     0.567568    0.148104       0.0        0.0      0.39558   
84  0.279085     0.702703    0.084123       0.0        0.0      0.39558   
64  0.342690     0.540541    0.069905       0.0        0.0      0.41947   

    Sentiment_Score  Tweets_Per_Day  Unique_Users  US_Viewers_In_Millions  
78         0.000000        0.171416      0.076392               4160000.0  
10         0.052632        0.012038      0.005178               5940000.0  
4          0.000000        0.007012      0.003157               6080000.0  
84         0.052632        0.147663      0.064629               4080000.0  
64         0.052632        0.169349      0.072055               7460000.0  


In [45]:
# Model
tf.logging.set_verbosity(tf.logging.ERROR)
regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols, 
                                          activation_fn = tf.nn.relu, hidden_units=[200, 100, 50, 25, 12])#,
                                         #optimizer = tf.train.GradientDescentOptimizer( learning_rate= 0.1 ))

In [46]:
# Reset the index of training
training_set.reset_index(drop = True, inplace =True)

In [47]:
def input_fn(data_set, pred = False):
    
    if pred == False:
        
        feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
        labels = tf.constant(data_set[LABEL].values)
        
        return feature_cols, labels

    if pred == True:
        feature_cols = {k: tf.constant(data_set[k].values) for k in FEATURES}
        
        return feature_cols

In [48]:
regressor.fit(input_fn=lambda: input_fn(training_set), steps=2000)
print("Done")



Done


In [49]:
# Evaluation on the test set created by train_test_split
ev = regressor.evaluate(input_fn=lambda: input_fn(testing_set), steps=1)
print(ev)
print("Done")

{'loss': 3624016800000.0, 'global_step': 2000}
Done


In [50]:
# Display the score on the testing set
# 0.002X in average
loss_score1 = ev["loss"]
print("Final Loss on the testing set: {0:f}".format(loss_score1))

Final Loss on the testing set: 3624016805888.000000


In [66]:
# Predictions
y = regressor.predict(input_fn=lambda: input_fn(testing_set))
print(list(y))

predictions = list(itertools.islice(y, testing_set.shape[0]))
print(type(y))


[4342217.5, 7331916.5, 6755606.5, 6244709.5, 5348859.5, 5674939.5, 6225052.0, 7452670.5, 5342902.5, 7329768.5, 5099173.0, 9320111.0, 7245795.5, 463777.22, 7352783.5, 4954099.0, 5496182.0, 4901030.0, 5440801.5, 8171113.0, 6936676.5, 6650694.0]
<class 'generator'>


In [64]:
predictions = pd.DataFrame(prepro_y.inverse_transform(np.array(predictions).reshape(22,1)),columns = ['Prediction'])

ValueError: cannot reshape array of size 0 into shape (22,1)

In [230]:
reality = pd.DataFrame(prepro.inverse_transform(testing_set), columns = [COLUMNS]).US_Viewers_In_Millions

In [231]:
matplotlib.rc('xtick', labelsize=30) 
matplotlib.rc('ytick', labelsize=30) 

fig, ax = plt.subplots(figsize=(50, 40))
plt.interactive(False)
plt.style.use('ggplot')
plt.plot(predictions.values, reality.values, 'ro')
plt.xlabel('Predictions', fontsize = 30)
plt.ylabel('Reality', fontsize = 30)
plt.title('Predictions x Reality on dataset Test', fontsize = 30)
ax.plot([reality.min(), reality.max()], [reality.min(), reality.max()], 'k--', lw=4)
plt.show()
print("Done")